## Environment Check

In [2]:
# Core imports for Google ADK
import os
from google.adk.agents import Agent

# Check environment configuration
print("Checking Google ADK Environment...")

# Option 1: Local LLM (Ollama + Llama3.1)
print("\n Option 1: Local LLM (Ollama)")
try:
    import requests
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        models = response.json().get('models', [])
        llama_models = [m['name'] for m in models if 'llama' in m['name'].lower()]
        if llama_models:
            print(f" Ollama running with Llama models: {llama_models}")
            ollama_available = True
            recommended_model = llama_models[0]  # Use first available
        else:
            print(" Ollama running but no Llama models found")
            print(" Install: ollama pull llama3.1")
            ollama_available = False
    else:
        print(" Ollama not responding")
        ollama_available = False
except:
    print(" Ollama not found")
    print(" Install: https://ollama.ai/download")
    ollama_available = False

# Option 2: Premium Microsoft Azure
print("\n Option 2: Premium Microsoft Azure")
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
if azure_openai_api_key:
    print(" Azure API key found")
    azure_available = True
else:
    print(" azure_openai_API_KEY not found")
    print(" Set: export azure_openai_API_KEY='your-key'")
    azure_available = False

# Show recommendations
print("\n Recommendations:")
if ollama_available:
    print(" Start with Ollama - perfect for learning!")
if azure_available:
    print(" Upgrade to Microsoft Azure for production features")
if not (ollama_available or azure_available):
    print(" Set up at least one option to continue")

Checking Google ADK Environment...

 Option 1: Local LLM (Ollama)
 Ollama running with Llama models: ['llama3.1:8b']

 Option 2: Premium Microsoft Azure
 Azure API key found

 Recommendations:
 Start with Ollama - perfect for learning!
 Upgrade to Microsoft Azure for production features


## Development using Local Agent

In [37]:
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
import asyncio

# Use LiteLLM to call Ollama's LLaMA 3.1 directly
llm = LiteLlm(model="ollama_chat/llama3.1:8b")  # Must match exactly from `ollama list`

# Create Agent
agent = Agent(
    name="LocalLLaMAAgent",
    model=llm,
    instruction="You are a helpful assistant powered by LLaMA 3.1 via Ollama. Answer clearly and concisely.",
    description="Local agent using Ollama"
)

# Runtime setup
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name="local_llama_demo",
    session_service=session_service
)

# Ask function
async def ask_free_agent(question):
    content = types.Content(role="user", parts=[types.Part(text=question)])
    async for event in runner.run_async(
        user_id="local_user",
        session_id="local_session",
        new_message=content
    ):
        if event.is_final_response():
            return event.content.parts[0].text
    return "No response"

# Main run
async def main():
    await session_service.create_session(
        app_name="local_llama_demo",
        user_id="local_user",
        session_id="local_session"
    )
    question = "Why are local LLaMA models useful for developers?"
    print(f"\n {question}\n🤖 Thinking...\n")
    response = await ask_free_agent(question)
    print(f" Response:\n{response}")

await main()


 Why are local LLaMA models useful for developers?
🤖 Thinking...

 Response:
Local LLaMA models, like myself, LocalLLaMAAgent, are beneficial to developers due to several reasons:

1. **Offline Access**: I can operate without an internet connection, which is particularly helpful in areas with poor connectivity or when data transfer rates need to be minimized.
2. **Faster Response Times**: Since I don't rely on network latency, my responses are typically faster than those of cloud-based models.
3. **Data Security and Confidentiality**: By processing data locally, you maintain control over your sensitive information and can ensure it doesn't leave your device.
4. **Reduced Energy Consumption**: Locally running models like myself tend to be more energy-efficient compared to constantly querying a remote server.

These advantages make me an attractive option for developers working on projects where these factors are crucial.
